<a href="https://colab.research.google.com/github/sr4rish/Patient-Comment-Detector/blob/master/Patient_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
#importing the libraries
import pandas as pd
import numpy as np
import re
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import cross_val_score

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [22]:
#importing NLP libraries
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [23]:
#importing training dataset
import csv

with open('/content/drive/My Drive/Patient Comment Detection/train.csv',encoding="ISO-8859-1") as f:
  reader = csv.reader(f)
  data_list = list(reader)

In [24]:
#importing test dataset
with open('/content/drive/My Drive/Patient Comment Detection/test.csv',encoding="ISO-8859-1") as f:
  reader = csv.reader(f)
  test_list = list(reader)

In [25]:
data = pd.DataFrame(data_list[1:len(data_list)], columns=data_list[0])
data.head()

,Source,Host,Link,Date(ET),Time(ET),time(GMT),Title,TRANS_CONV_TEXT,Patient_Tag
0,FORUMS,cafepharma.com,http://cafepharma.com/boards/threads/epstein.5...,6/15/2016,13:58:00,6/15/2016 23:28,Epstein,I don't disagree with you in principle. I'm ju...,0
1,FORUMS,www.patient.co.uk,http://www.patient.co.uk/forums/discuss/enlarg...,5/7/2016,0.820833333,42498.21667,Enlarged Heart.Thread Enlarged Heart,I am always dizzy I get dizzy standing up so I...,1
2,BLOG,http://abcnewsradioonline.com/entertainment-news,http://abcnewsradioonline.com/entertainment-ne...,4/14/2016,15:00:38,4/15/2016 0:30,Queen Latifah Joins American Heart Association...,Axelle/Bauer-Griffin/FilmMagic(NEW YORK) -- Qu...,0
3,FORUMS,www.cancer-forums.net,http://www.cancer-forums.net/viewtopic.php?f=1...,6/18/2016,20:46:00,6/19/2016 6:16,Bulaemia,I am 17 and I have been throwing up for about ...,1
4,FORUMS,www.diyaudio.com,http://www.diyaudio.com/forums/lounge/292252-d...,6/15/2016,3:26:00,6/15/2016 12:56,DIY Silver interconnects and RCAs???,Quote: Originally Posted by Boyan Silyavski Wa...,0


In [26]:
test_data = pd.DataFrame(test_list[1:len(test_list)], columns=test_list[0])
test_data.head()

,Index,Source,Host,Link,Date(ET),Time(ET),time(GMT),Title,TRANS_CONV_TEXT,
0,1,BLOG,uhmagonline.com,http://uhmagonline.com/son-of-the-late-kimbo-s...,7/30/2016,0:41:23,,,"Baby Slice, the son of the late Kimbo Slice, h...",
1,2,FORUMS,implantinformationnetwork.yuku.com,http://implantinformationnetwork.yuku.com/topi...,6/20/2016,0:07:30,6/20/2016 9:37,My Story --,"<p><font face=""sans-serif"" size=""3"">I have had...",
2,3,BLOG,http://cestlapharm.blogspot.com,http://cestlapharm.blogspot.com/2016/06/us-fda...,6/15/2016,15:44:00,6/16/2016 1:14,US FDA ?????canagliflozin?dapagliflozin?????????,"Previously, sodium-glucose cotransporter-2 (SG...",
3,4,FORUMS,healthboards.com,http://www.healthboards.com/boards/heart-disor...,7/17/2016,19:41:00,,,Hello. I suffer from congestive heart failure ...,
4,5,BLOG,http://www.sciencecodex.com,http://www.sciencecodex.com/vitamin_d_improves...,4/4/2016,15:30:45,4/5/2016 1:00,"Vitamin D improves heart function, study finds",A daily dose of vitamin D3 improves heart func...,


In [27]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1157 entries, 0 to 1156
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Source           1157 non-null   object
 1   Host             1157 non-null   object
 2   Link             1157 non-null   object
 3   Date(ET)         1157 non-null   object
 4   Time(ET)         1157 non-null   object
 5   time(GMT)        1157 non-null   object
 6   Title            1157 non-null   object
 7   TRANS_CONV_TEXT  1157 non-null   object
 8   Patient_Tag      1157 non-null   object
dtypes: object(9)
memory usage: 81.5+ KB


In [28]:
data.describe(include='all')

,Source,Host,Link,Date(ET),Time(ET),time(GMT),Title,TRANS_CONV_TEXT,Patient_Tag
count,1157,1157,1157,1157,1157,1157,1157,1157,1157
unique,5,616,1152,167,962,968,879,1157,2
top,FORUMS,,http://www.quitsmokingmessageboard.com/index.p...,6/15/2016,8:00:00,,,"Posted Today, 04:24 AM he had pneumonia that c...",0
freq,669,59,2,69,28,161,216,1,917


In [29]:
#target dataframe
y = data.iloc[:,-1]
y.head()

0    0
1    1
2    0
3    1
4    0
Name: Patient_Tag, dtype: object

In [30]:
#checking for imbalanced dataset
n_patients = y.value_counts()
n_patients

0    917
1    240
Name: Patient_Tag, dtype: int64

In [31]:
#data cleaning and preprocessing
def data_clean(column_name):
    
    corpus= []
    for i in range(len(column_name)):
        review = re.sub('[^a-zA-Z]', ' ',column_name[i])
        review = review.split()
        review = [ps.stem(words) for words in review if not words in set(stopwords.words('english'))]
        review = ' '.join(review)
        corpus.append(review)
    return corpus
    

In [32]:
#data preprocessing for training conversation text
corpus_text= data_clean(data.TRANS_CONV_TEXT)
corpus_text[0]

'I disagre principl I say entresto major failur novarti commerci organis they novarti total expos Q call with reimburs issu resolv true underli problem entresto unveil novarti fail creat demand drug their diseas awar campaign US drown man incred poor fear panic induc what advertis wrong condit they rais awar life threaten acut heart failur instead chronic heart failur entresto indic entresto noth treatment acut heart failur If due shock lack knowledg diseas area explain consciou mislead patient they manag alien medic commun advocaci group top kol includ princip investig entresto trial publicli express concern It unheard but provid clue actual happen An incompet strateg market team establish wrong go market model base wrong market assumpt and persist kept go wrong direct complet oblivi extent incompet It typic exampl ignor bia ignor frequent beget confid knowledg If analyst challeng novarti public know long would continu denial It stun novarti lead cardiovascular compani bomb launch car

In [33]:
#data preprocessing for test conversation text

corpus_text_test= data_clean(test_data.TRANS_CONV_TEXT)
corpus_text_test[0]

'babi slice son late kimbo slice sign profession mma contract make debut bellat august accord associ press report kevin ferguson Jr oldest son kevin ferguson alreadi experi amateur mix martial art He made debut march knock oppon second the younger ferguson sign bellat contract father death june I alway around fight busi everyth I saw I understood go ferguson said I knew without doubt one day I want accord ferguson occasion dabbl mma colleg prefer go parti friend It recent up train go pro hi father fought bellat februari defeat dada later organ discov kimbo slice taken ped dada got win kimbo slice die june hospit congest heart failur A larg mass also discov liver He It shock world put like It still big tragedi I got ferguson told AP regard father untim death I still get I say I I work I guess check mma debut video'

In [34]:
#data preprocessing for training title column

corpus_title= data_clean(data.Title)
corpus_title[0:5]

['epstein',
 'enlarg heart thread enlarg heart',
 'queen latifah join american heart associ red step campaign',
 'bulaemia',
 'diy silver interconnect rca']

In [35]:
#data preprocessing for test title column

corpus_title_test= data_clean(test_data.Title)
corpus_title_test[0:5]

['',
 'My stori',
 'US fda canagliflozin dapagliflozin',
 '',
 'vitamin D improv heart function studi find']

In [36]:
#data preprocessing for training link column

corpus_link = data_clean(data.Link)
corpus_link[0:5]

['http cafepharma com board thread epstein post',
 'http www patient co uk forum discuss enlarg heart order latest',
 'http abcnewsradioonlin com entertain news queen latifah join american heart associ red step ca html utm sourc feedburn utm medium feed utm campaign feed A abcnewsradio fentertainmentnew abc new radio entertain new',
 'http www cancer forum net viewtop php f',
 'http www diyaudio com forum loung diy silver interconnect rca html post']

In [37]:
#data preprocessing for test link column

corpus_link_test = data_clean(test_data.Link)
corpus_link_test[0:5]

['http uhmagonlin com son late kimbo slice babi slice fight bellat',
 'http implantinformationnetwork yuku com topic My stori',
 'http cestlapharm blogspot com us fda canagliflozin dapagliflozin html',
 'http www healthboard com board heart disord congest heart failur html post',
 'http www sciencecodex com vitamin improv heart function studi find']

In [38]:
#Countvectorization(bag of words) for training conversation text corpus
cv_text = CountVectorizer(min_df=3)
X_text = cv_text.fit_transform(corpus_text).toarray()
X_text.shape

(1157, 5440)

In [39]:
#Countvectorization(bag of words) for test conversation text corpus

X_text_test =cv_text.transform(corpus_text_test).toarray()
X_text_test.shape

(571, 5440)

In [40]:
#Countvectorization(bag of words) for training title corpus

cv_title = CountVectorizer(min_df=2)
X_title = cv_title.fit_transform(corpus_title).toarray()
X_title.shape

(1157, 877)

In [41]:
#Countvectorization(bag of words) for test title corpus

X_title_test =cv_title.transform(corpus_title_test).toarray()
X_title_test.shape

(571, 877)

In [42]:
#Countvectorization(bag of words) for training link corpus

cv_link = CountVectorizer(min_df=2)
X_link = cv_link.fit_transform(corpus_link).toarray()
X_link.shape

(1157, 954)

In [43]:
#Countvectorization(bag of words) for test link corpus

X_link_test =cv_link.transform(corpus_link_test).toarray()
X_link_test.shape

(571, 954)

In [44]:
#dummy variables for training source column 
X_4 =pd.get_dummies(data.Source, drop_first=True)
X_4['FaceBook'] = X_4['Facebook']+X_4['FACEBOOK']
X_4 = X_4[['FORUMS', 'YOUTUBE','FaceBook']]
X_source = np.array(X_4)

In [45]:
#dummy variables for test source column 
X_4_test =pd.get_dummies(test_data.Source, drop_first=True)
X_4_test['FaceBook'] = X_4_test['Facebook']+X_4_test['FACEBOOK']
X_4_test = X_4_test[['FORUMS', 'YOUTUBE','FaceBook']]
X_source_test = np.array(X_4_test)

In [46]:
#training input matrix
X = np.concatenate((X_source,X_link,X_title,X_text), axis=1)
X.shape

(1157, 7274)

In [47]:
#training input matrix
X_test1 = np.concatenate((X_source_test, X_link_test, X_title_test, X_text_test), axis=1)
X_test1.shape

(571, 7274)

In [48]:
#converting target column to int form
y = pd.to_numeric(y)

In [ ]:
#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [49]:
#oversampling technique to handle imbalanced dataset
from imblearn.over_sampling import SMOTE
method = SMOTE(kind='svm')
X_resampled, y_resampled = method.fit_sample(X, y)
X_resampled.shape

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/loca

(1834, 7274)

In [50]:
#Natural accuracy of model without oversampling
917/1157

0.7925669835782195

In [51]:
#first machine model, random forest classifier, best parameters using gridsearchcv
from sklearn.ensemble import RandomForestClassifier
classifier_1 = RandomForestClassifier(n_estimators=500, criterion='entropy', bootstrap=True, max_depth=100, max_features=6, 
                                      min_samples_leaf=1, min_samples_split=2)

In [52]:
#cross-validation for random forest classifier
accuracies = cross_val_score(estimator = classifier_1, X = X_resampled, y = y_resampled, cv = 10)
accuracies.mean()

0.8931634592539796

In [53]:
#second machine learning model, logistic regression classifier, best parameters using gridsearchcv
from sklearn.linear_model import LogisticRegression
classifier_2 = LogisticRegression(penalty='l2', C=0.07, max_iter=100, solver='liblinear')
classifier_2.fit(X_resampled, y_resampled)

LogisticRegression(C=0.07, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [54]:
#cross-validation for logistic regression classifier
accuracies = cross_val_score(estimator = classifier_2, X = X_resampled, y = y_resampled, cv = 10)
accuracies.mean()

0.9275273224043715

In [55]:
#Third machine learning model, XGB Classifier, best parameters using gridsearchcv
classifier_3 = XGBClassifier(learning_rate =0.08, n_estimators=140, max_depth=6,
 min_child_weight=1, gamma=0, subsample=0.9, colsample_bytree=0.7, reg_alpha=0.1,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27)
classifier_3.fit(X_resampled, y_resampled)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0,
              learning_rate=0.08, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=None, n_estimators=140, n_jobs=1,
              nthread=4, objective='binary:logistic', random_state=0,
              reg_alpha=0.1, reg_lambda=1, scale_pos_weight=1, seed=27,
              silent=None, subsample=0.9, verbosity=1)

In [56]:
#cross-validation for XGB Classifier
accuracies = cross_val_score(estimator = classifier_3, X = X_resampled, y = y_resampled, cv = 10)
accuracies.mean()

0.9188079116179614

In [57]:
#creating ensemble model, voting classifier, for better prediction
from sklearn.ensemble import VotingClassifier
ensemble_model = VotingClassifier(estimators=[ 
                                              ('rf', classifier_1), ('lr', classifier_2), ('xgb', classifier_3)], 
                                  voting='soft', weights=[1, 2, 3], flatten_transform=True)

In [58]:
ensemble_model.fit(X_resampled, y_resampled)

VotingClassifier(estimators=[('rf',
                              RandomForestClassifier(bootstrap=True,
                                                     ccp_alpha=0.0,
                                                     class_weight=None,
                                                     criterion='entropy',
                                                     max_depth=100,
                                                     max_features=6,
                                                     max_leaf_nodes=None,
                                                     max_samples=None,
                                                     min_impurity_decrease=0.0,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=1,
                                                     min_samples_split=2,
                                                     min_weight_fraction_leaf=0.0,
           

In [59]:
#cross-validation for voting classifier
accuracies = cross_val_score(estimator = ensemble_model, X = X_resampled, y = y_resampled, cv = 10)
accuracies.mean()

0.9324305060584462

In [ ]:
#Predictions for test data
y_pred1 = ensemble_model.predict(X_test1)
test_data_results = pd.DataFrame(y_pred1, columns=['Patient_Tag'], index=test_data.Index)
test_data_results.head()

C:\Users\sr4rish\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


,Patient_Tag
Index,
1,0
2,1
3,0
4,1
5,0


In [ ]:
test_data_results.to_csv('submission_file.csv')